In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
import re
import pandas as pd
from NedbankSpendPy import  client
import time

In [2]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [3]:
def client_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{'companyname':1,
                                                                    'Seg_L2_STR':1,
                                                                    'companyindex':1,
                                                                   'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
                                                                   'companyindex':'$companyindex'
                                                               },
                                                               "uniqueclientlist":{"$addToSet":'$Dedupegroup'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'companyindex':'$_id.companyindex',
                                                                    'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [27]:
def company_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            },
#                                                             'Seg_L2_Num':1 # Only look at the BOE segment!                                              
                                                        }
                                                       },
                                                       {'$project':{
                                                           'companyname':1,
                                                           'companyindex':1,
                                                           'Seg_L2_STR':1,
                                                           'Seg_L2_Num':1,
#                                                            'franchisename':1,
                                                           'Dedupegroup':1,
#                                                            'companyindex':1,
                                                           'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
                                                                   'companyindex':'$companyindex',
                                                                   'Seg_L2_STR':'$Seg_L2_STR',
                                                                   'Seg_L2_Num':'$Seg_L2_Num'
#                                                                    'franchisename':'$franchisename'
                                                               },
                                                               "uniqueclientlist":{"$addToSet":'$Dedupegroup'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'Seg_L2_STR':'$_id.Seg_L2_STR',
                                                                    'Seg_L2_Num':'$_id.Seg_L2_Num',
                                                                    'companyindex':'$_id.companyindex',
                                                                    'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [36]:
collectionlist=['clientswipes_201801','clientswipes_201802','clientswipes_201803',
                'clientswipes_201804','clientswipes_201805','clientswipes_201806',
                'clientswipes_201807','clientswipes_201808','clientswipes_201809',
                'clientswipes_201810','clientswipes_201811','clientswipes_201812',                
                'clientswipes_201901','clientswipes_201902','clientswipes_201903',
                'clientswipes_201904','clientswipes_201905','clientswipes_201906',
                'clientswipes_201907','clientswipes_201908','clientswipes_201909',
                'clientswipes_201910','clientswipes_201911','clientswipes_201912',
               'clientswipes_202001','clientswipes_202002']

In [37]:
from datetime import datetime 

In [38]:
# collectionname=collectionlist[0]
# collection=client.behaviourdb[collectionname]
# df=franchise_agg_fn(collection)

In [39]:
savestr

'20200407'

In [41]:
dn=datetime.now()
m=dn.month
d=dn.day
y=dn.year
savestr= f'{y:0}'+f'{m:02}'+f'{d:02}'
write_collection = client.lookups['aggregated_company_spend']
# df=pd.DataFrame()
collectionlist=['clientswipes_202001','clientswipes_202002']
for collectionname in collectionlist:
#     print('-----------------------------------------------------------------------------')
    print('===== now doing collection: {collectionname} ====================================='.format(collectionname=collectionname))
    period=collectionname[-6:]
    print('Doing period: {period}'.format(period=period))
    collection=client.behaviourdb[collectionname]
    print('Collect data:')    
    agg_df=company_agg_fn(collection)
    print('Set index:')    
    agg_df.set_index(['companyindex','companyname'],inplace=True)
    print('Rename columns:')        
    agg_df.rename(columns={'transactionamount':'amount_{period}'.format(period=period),
                      'transactioncount':'count_{period}'.format(period=period),
                          'n_uniqueclients':'n_uniqueclients_{period}'.format(period=period)}, inplace=True)
#     agg_df['period']=period
#     if df.empty:
#         print('The history is still empty!')
#         df=agg_df.copy()
#     else:
#         print('Merge new data with (non-empty) history:')        
#         df=df.merge(agg_df, right_index=True,left_index=True, how='outer')
#     print('Save (new) history:')
#     print('Save (new) history:')
#     agg_df.reset_index(inplace=True)
    agg_df['extracted_date']=savestr
    agg_df.reset_index(inplace=True)
    write_collection.insert_many(agg_df.to_dict('records'))
#     df.to_csv(df.to_csv('/home/christo/data/companydb_franchiselevel_aggregated_pivot_{savestr}.csv'.format(savestr=savestr)))
    

===== now doing collection: clientswipes_202001 =====================================
Doing period: 202001
Collect data:
Set index:
Rename columns:
===== now doing collection: clientswipes_202002 =====================================
Doing period: 202002
Collect data:
Set index:


KeyError: 'companyindex'

In [33]:
agg_df.reset_index(inplace=True)

In [42]:
agg_df.shape

(0, 0)

In [ ]:
df.to_csv('/home/christo/data/companydb_franchiselevel_aggregated_pivot_{savestr}.csv'.format(savestr=savestr))

In [ ]:
collection = client.lookups['aggregated_company_spend']
df.reset_index(inplace=True)
df['extracted_date']=savestr
collection.insert_many(df.to_dict('records'))

In [ ]:
list(df)

#### Extract distinct franchisenames:

#### - BOLT: